# RAG Workflow & Backlog

## Overview & Goals

**Challenge:** Build a production-ready Retrieval-Augmented Generation (RAG) system for Deutsche Telekom that enables question-answering over internal publications and documents.

**Goals:**
- Efficiently index and retrieve relevant document chunks
- Generate accurate, cited responses using retrieved context
- Maintain traceability with publication IDs
- Support enterprise requirements (security, monitoring, scalability)
- Enable easy experimentation and iteration

**Key Components:**
- Document loading with metadata extraction
- Metadata-aware chunking
- Vector-based retrieval with optional reranking
- LLM-based generation with citation
- Structured logging and observability


## Data Exploration

Load and examine sample documents from the data folder.


In [ ]:
import sys
from pathlib import Path

# Add project root to path (notebooks/ is one level below project root)
project_root = Path().resolve().parent
if (project_root / "config").exists():
    sys.path.insert(0, str(project_root))
else:
    # Try current directory if already at project root
    if (Path().resolve() / "config").exists():
        sys.path.insert(0, str(Path().resolve()))

from data.loader import DocumentLoader
from config import settings

# Load a few sample documents
loader = DocumentLoader(data_folder=settings.DATA_FOLDER)
documents = loader.load_all_documents()

print(f"Total documents loaded: {len(documents)}")
print(f"\nSample documents:")
for i, doc in enumerate(documents[:3], 1):
    pub_id = doc.metadata.get("publication_id", "unknown")
    word_count = doc.metadata.get("word_count", len(doc.page_content.split()))
    print(f"\n{i}. Publication ID: {pub_id}")
    print(f"   Word count: {word_count}")
    print(f"   Topics: {doc.metadata.get('topics', [])}")
    print(f"   Preview: {doc.page_content[:200]}...")


In [ ]:
# Word statistics across all documents
if documents:
    total_words = sum(doc.metadata.get("word_count", 0) for doc in documents)
    avg_words = total_words / len(documents) if documents else 0
    
    print(f"\nDocument Statistics:")
    print(f"  Total documents: {len(documents)}")
    print(f"  Total words: {total_words:,}")
    print(f"  Average words per document: {avg_words:.0f}")
    print(f"  Documents with topics: {sum(1 for d in documents if d.metadata.get('topics'))}")
    print(f"  Documents with dates: {sum(1 for d in documents if d.metadata.get('mentioned_dates'))}")
    
    # Count by topics
    all_topics = []
    for doc in documents:
        all_topics.extend(doc.metadata.get("topics", []))
    
    from collections import Counter
    topic_counts = Counter(all_topics)
    print(f"\nTopic distribution:")
    for topic, count in topic_counts.most_common():
        print(f"  {topic}: {count}")


## Indexing Workflow

Index documents using the loader, chunker, and ChromaDB (in-memory for notebook).


In [ ]:
from core.chunking import MetadataAwareChunker
from core.embeddings import get_embeddings
from langchain.vectorstores import Chroma

# Get embeddings
embeddings = get_embeddings()
print("✓ Loaded embeddings model")

# Chunk documents with metadata
chunker = MetadataAwareChunker(
    chunk_size=settings.CHUNK_SIZE,
    chunk_overlap=settings.CHUNK_OVERLAP
)
print(f"✓ Initialized chunker (size={settings.CHUNK_SIZE}, overlap={settings.CHUNK_OVERLAP})")

# Chunk all documents
all_chunks = []
for doc in documents[:5]:  # Use first 5 docs for demo
    source = doc.metadata.get("source", "Deutsche Telekom")
    doc_id = doc.metadata.get("publication_id", doc.metadata.get("file_name", "unknown"))
    extra_metadata = {
        k: v for k, v in doc.metadata.items()
        if k not in ["source", "publication_id", "file_name"]
    }
    
    chunks = chunker.chunk_with_metadata(
        text=doc.page_content,
        source=source,
        doc_id=doc_id,
        **extra_metadata
    )
    all_chunks.extend(chunks)

print(f"✓ Created {len(all_chunks)} chunks from {min(5, len(documents))} documents")
print(f"  Average chunks per document: {len(all_chunks) / min(5, len(documents)):.1f}")

# Show sample chunk metadata
if all_chunks:
    sample = all_chunks[0]
    print(f"\nSample chunk metadata:")
    print(f"  chunk_id: {sample.metadata.get('chunk_id')}")
    print(f"  doc_id: {sample.metadata.get('doc_id')}")
    print(f"  chunk_index: {sample.metadata.get('chunk_index')}")
    print(f"  total_chunks: {sample.metadata.get('total_chunks')}")


In [ ]:
# Create in-memory ChromaDB (no persistence for notebook)
vectordb = Chroma.from_documents(
    documents=all_chunks,
    embedding=embeddings,
)

print(f"✓ Indexed {len(all_chunks)} chunks into ChromaDB")
print(f"  Vector dimensions: {len(embeddings.embed_query('test'))}")
print(f"  Collection count: {vectordb._collection.count() if hasattr(vectordb, '_collection') else 'N/A'}")


## Retrieval Demo

Run queries and examine retrieved sources.


In [ ]:
from core.retrieval import AdvancedRetriever

# Create retriever (no reranker for CPU-only demo)
retriever = AdvancedRetriever(vectordb=vectordb, reranker_model=None)

# Test queries
queries = [
    "What is 5G?",
    "Tell me about security",
    "Partnership information"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    results = retriever.retrieve(query=query, top_k=3)
    
    print(f"Retrieved {len(results)} documents:\n")
    for i, doc in enumerate(results, 1):
        pub_id = doc.metadata.get("publication_id", doc.metadata.get("doc_id", "unknown"))
        chunk_idx = doc.metadata.get("chunk_index", "N/A")
        print(f"{i}. [Publication: {pub_id}, Chunk: {chunk_idx}]")
        print(f"   {doc.page_content[:150]}...")
        print()


## Generation Demo

Build prompts and generate responses. Using mock model for CPU-only execution.


In [ ]:
from llm.prompt_manager import PromptManager

prompt_manager = PromptManager()

# Build RAG prompt with retrieved context
query = "What services does Deutsche Telekom offer?"
retrieved_docs = retriever.retrieve(query=query, top_k=2)

prompt = prompt_manager.build_rag_prompt(
    query=query,
    context_docs=retrieved_docs,
    chat_history=None
)

print("Generated Prompt:")
print("="*80)
print(prompt[:1000])  # Show first 1000 chars
print("...")
print("="*80)
print(f"\nPrompt length: {len(prompt)} characters")
print(f"Context documents used: {len(retrieved_docs)}")


In [ ]:
# Mock generation for CPU-only demo (no actual LLM call)
# In production, this would call generate_response() with real model

print("Mock Generation Response:")
print("="*80)

# Extract publication IDs from sources
publication_ids = []
for doc in retrieved_docs:
    pub_id = doc.metadata.get("publication_id", doc.metadata.get("doc_id", "unknown"))
    if pub_id not in publication_ids:
        publication_ids.append(pub_id)

# Simulate response (in real system, this comes from LLM)
mock_response = f"""Based on the retrieved documents, Deutsche Telekom offers various telecommunications services including 5G network infrastructure, secure cloud solutions, and enterprise partnerships. The company focuses on expanding coverage and providing reliable connectivity services.

Sources: {', '.join(publication_ids)}"""

print(mock_response)
print("="*80)
print(f"\n[Note: This is a mock response. In production, use generate_response() with loaded model]")


## Observability

Show structured logging output example.


In [ ]:
from monitoring.logging import StructuredLogger
import json

# Create logger
logger = StructuredLogger("notebook_demo", log_dir="/tmp/notebook_logs")

# Log a sample query
sample_docs = retrieved_docs[:2] if retrieved_docs else []
logger.log_query(
    query="What services does Deutsche Telekom offer?",
    retrieved_docs=sample_docs,
    response_time=1.23,
    user_id="notebook_user"
)

# Show what was logged (read the JSON line from the log file)
import os
log_file = "/tmp/notebook_logs/notebook_demo.jsonl"
if os.path.exists(log_file):
    with open(log_file, "r") as f:
        log_line = f.readlines()[-1] if f.readlines() else None
        if log_line:
            log_data = json.loads(log_line)
            print("Sample Structured Log Entry:")
            print("="*80)
            print(json.dumps(log_data, indent=2))
            print("="*80)
else:
    print("Sample log entry structure:")
    sample_log = {
        "timestamp": "2024-01-01T12:00:00Z",
        "level": "INFO",
        "logger": "notebook_demo",
        "message": "Query processed",
        "event_type": "query",
        "query": "What services does Deutsche Telekom offer?",
        "num_documents": 2,
        "response_time_seconds": 1.23,
        "user_id": "notebook_user",
        "document_ids": ["doc_1", "doc_2"]
    }
    print(json.dumps(sample_log, indent=2))


## Backlog of Ideas

Future enhancements and experiments for the RAG system:

### Retrieval & Ranking
- **Switch to better embeddings**: Evaluate and switch to `intfloat/e5-large-v2` or `bge-large-en-v1.5` embeddings; compare recall@k metrics against current `all-MiniLM-L6-v2`
- **Max Marginal Relevance (MMR)**: Implement diversity-based retrieval to reduce redundant results
- **Hybrid Retrieval**: Combine sparse+dense retrieval (BM25 + vector search) for improved recall
- **Reranker A/B Testing**: Compare `cross-encoder/ms-marco-MiniLM-L-6-v2` vs `bge-reranker-large`; log reranking scores for analysis

### Provenance & Citations
- **Normalized chunk IDs + provenance**: Store byte spans and filenames for each chunk
- **Inline snippet highlights**: Show highlighted source snippets in responses with exact character spans

### Response Quality & Guardrails
- **Response guardrails**: 
  - Require citations for factual statements
  - Refusal mechanism when confidence score is below threshold
  - Log confidence scores for monitoring
- **Prompt compression (LLMLingua)**: Compress long context for better efficiency while preserving key information

### Evaluation & Quality
- **Eval harness**: Implement RAGAS (Retrieval-Augmented Generation Assessment) metrics:
  - Context Precision
  - Context Recall  
  - Faithfulness
  - Answer Relevancy
- **Hand-curated Q/A set**: Create golden dataset over the 250 documents for regression testing
- **Nightly CI evaluation**: Run RAGAS metrics in CI pipeline on golden set to detect regressions

### Performance & Optimization
- **Caching**: Implement Redis/SQLite caching layer keyed by `(normalized_query, embedding_hash, index_version)` to avoid redundant retrievals and generations
- **Vector DB swaps**: Support FAISS index via Chroma settings; create plug-in layer for Milvus/PGVector backends
- **Async Processing**: Use async/await for concurrent operations

### User Experience
- **Cancellation support**: Allow users to cancel long-running queries
- **Queue limit**: Implement request queue limits to prevent overload

### Security & Authentication
- **FastAPI authentication**: Implement API key or OAuth2 authentication
- **Rate limiting**: Add per-user/IP rate limiting to prevent abuse

### Security & Authentication
- **FastAPI authentication**: Implement API key or OAuth2 authentication
- **Rate limiting**: Add per-user/IP rate limiting to prevent abuse

### Multilingual & Internationalization
- **Multilingual Support**: Support documents in multiple languages with language detection
- **Cross-lingual Retrieval**: Retrieve relevant content even when query language differs from document language

### Data Management & Deployment

- **Externalize data folder**: Inject the data/ directory at runtime instead of committing it to Git:
  - Mount it as a Docker/K8s volume or download from secure storage (S3, Azure Blob, etc.) during startup.
  - Add checksum validation and update scripts to ensure dataset integrity across environments.